In [1]:
from flask import Flask, request, jsonify
import joblib
import json
import numpy as np
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder, LabelBinarizer, OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
import dill

/var/folders/dm/xc68z2ls0d5gs2zbxxn7vn680000gn/T/ipykernel_23376/3119173143.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Cargar el modelo entrenado
modelo_pred = joblib.load('modelo_entrenado.pkl')
modelo_clus = joblib.load('modelo_clustering_entrenado.pkl')

with open('pipeline.pkl', 'rb') as f:
    pipeline = dill.load(f)

In [3]:
# Inicializar la aplicación Flask
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():

    # Conseguir los datos de la petición
    data = request.get_json(force=True)

    # Convertir el JSON a un diccionario de Python
    datos_pred_dict = json.loads(data)

    # Crear un DataFrame a partir del diccionario
    parapred = pd.DataFrame([datos_pred_dict])

    # Hacer la predicción
    data_transformed = pipeline.transform(parapred)
    
    prediction = modelo_pred.predict(data_transformed)
    cluster = modelo_clus.predict(data_transformed)

    # Convertir la predicción a una lista para que sea serializable a JSON
    prediction_list = prediction.tolist()
    cluster_list = cluster.tolist()

    return jsonify({"prediction": prediction_list, "cluster": cluster_list})

In [4]:
if __name__ == '__main__':
    # Ejecutar la aplicación Flask
    app.run(port=8000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [17/Sep/2024 08:37:18] "POST /predict HTTP/1.1" 200 -
